In [3]:
# wfdb.dl_database("mitdb", "database", "all")   НЕ НАЖИМАТЬ!!!!!!!!

In [2]:
import torch
import torch.nn as nn
import wfdb
import numpy as np

In [3]:
# Загрузка данных с помощью wfdb
record = wfdb.rdrecord('database/232')
# record = wfdb.rdrecord('database/213')
data = record.p_signal[:, 0] # берем только первый сигнал
# print(data)
print(len(data))
# annotations = wfdb.rdann('database/213', 'dat')
annotations = wfdb.rdann('database/232', 'dat')
print(annotations.symbol)
print(len(annotations.symbol))

650000
['|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'S', '|', nan, 'S', '|', nan, 'E', '|', nan, 'j', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'E', '|', nan, 'j', '|', nan, 'j', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'S', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'E', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'j', '|', nan, '/', '|', nan, 'j', '|', nan, 'E', '|', nan, 'E', '|', nan, 'E', '|', nan, 'j', '|', nan, '/', '|', nan, 'E', '|', nan, 'E', '|', nan, 'j', '|', nan, 'j', '|', nan, 'j', '|', nan, '/', '|', nan, '/', '|', nan, 'E', '|', nan, 'E', '|', nan, 'j', '|', nan, '/', '|', nan, 'j', '|', nan, 'E', '|', nan, 'S', '|', nan, 'S', '|', nan, 'S', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'A', '|', nan, 'S', '|', nan, 'E', '|', nan, 'j', '|', nan, 'j', '|', nan, 'E', '|', nan, 'S', '|', nan, 'S', '|', nan, 'E', '|

In [9]:
# Подготовка данных для обучения нейронной сети
window_size = 100 # размер окна
step_size = 1 # шаг окна
x_train = []
y_train = []
# for i in range(0, len(data) - window_size, step_size):
for i in range(0, len(annotations.symbol), step_size):
    x_train.append(data[i:i+window_size])
    if annotations.symbol[i] in ['N', 'L', 'R', 'A', 'a', 'J', 'S', 'V', 'F', 'e', 'j', 'E', '/', 'f', 'Q', '?']:
        y_train.append(0) # здоровый сигнал
    else:
        y_train.append(1) # сигнал с болезнью
# print(y_train)
print(len(y_train))

556102


In [10]:
# Преобразование данных в формат PyTorch Tensor
x_train = torch.Tensor(x_train).unsqueeze(1)
y_train = torch.Tensor(y_train).unsqueeze(1)

# Определение архитектуры рекуррентной нейронной сети
class RNN(nn.Module):
    def __init__(self, 
                 input_size, 
                 hidden_size, 
                 output_size):
        
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, hidden = self.rnn(x)
        hidden = hidden.view(-1, self.hidden_size)
        output = self.fc(hidden)
        return output

# Обучение нейронной сети на подготовленных данных
input_size = window_size
hidden_size = 100
output_size = 1
learning_rate = 0.001
criterion = nn.BCEWithLogitsLoss() # функция потерь
rnn = RNN(input_size, hidden_size, output_size)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate) # оптимизатор

for epoch in range(100):
    optimizer.zero_grad()
    outputs = rnn(x_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    print('Epoch {}, Loss: {:.4f}'.format(epoch+1, loss.item()))

# Тестирование нейронной сети на тестовых данных
x_test = []
y_test = []
for i in range(0, len(annotations.symbol), step_size):
    x_test.append(data[i:i+window_size])
    if annotations.symbol[i] in ['N', 'L', 'R', 'A', 'a', 'J', 'S', 'V', 'F', 'e', 'j', 'E', '/', 'f', 'Q', '?']:
        y_test.append(0) # здоровый сигнал
    else:
        y_test.append(1) # сигнал с болезнью

# Преобразование данных в формат PyTorch Tensor
x_test = torch.Tensor(x_test).unsqueeze(1)
y_test = torch.Tensor(y_test).unsqueeze(1)

# Оценка качества модели
with torch.no_grad():
    outputs = rnn(x_test)
    predicted = outputs.round()
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print((predicted == y_test).sum(), len(y_test))
    print((predicted == y_test).sum().item(), len(y_test))
    print("Accuracy:", accuracy)

Epoch 1, Loss: 0.7102
Epoch 2, Loss: 0.6950
Epoch 3, Loss: 0.6826
Epoch 4, Loss: 0.6729
Epoch 5, Loss: 0.6659
Epoch 6, Loss: 0.6614
Epoch 7, Loss: 0.6589
Epoch 8, Loss: 0.6582
Epoch 9, Loss: 0.6587
Epoch 10, Loss: 0.6598
Epoch 11, Loss: 0.6609
Epoch 12, Loss: 0.6619
Epoch 13, Loss: 0.6623
Epoch 14, Loss: 0.6623
Epoch 15, Loss: 0.6617
Epoch 16, Loss: 0.6608
Epoch 17, Loss: 0.6596
Epoch 18, Loss: 0.6583
Epoch 19, Loss: 0.6571
Epoch 20, Loss: 0.6560
Epoch 21, Loss: 0.6551
Epoch 22, Loss: 0.6544
Epoch 23, Loss: 0.6540
Epoch 24, Loss: 0.6537
Epoch 25, Loss: 0.6536
Epoch 26, Loss: 0.6536
Epoch 27, Loss: 0.6536
Epoch 28, Loss: 0.6535
Epoch 29, Loss: 0.6534
Epoch 30, Loss: 0.6532
Epoch 31, Loss: 0.6529
Epoch 32, Loss: 0.6526
Epoch 33, Loss: 0.6521
Epoch 34, Loss: 0.6517
Epoch 35, Loss: 0.6513
Epoch 36, Loss: 0.6509
Epoch 37, Loss: 0.6506
Epoch 38, Loss: 0.6503
Epoch 39, Loss: 0.6501
Epoch 40, Loss: 0.6499
Epoch 41, Loss: 0.6498
Epoch 42, Loss: 0.6497
Epoch 43, Loss: 0.6495
Epoch 44, Loss: 0.64